# Part III – Prompt-Engineering Extension (Parameterization)

## Extension Overview
This notebook implements a **parameterization extension** of the tutorial `sequence_detector` example from `ChipChat.ipynb`.  
The original fixed-length sequence detector was refactored into a **configurable, synthesizable RTL module** that supports variable pattern length, pattern value, and enable-gated operation.

**Extension type chosen:** Parameterization  
**Base tutorial example:** `sequence_detector` (FSM / sequential logic)

#1) Install tools (iverilog)

In [23]:
!apt-get update -y
!apt-get install -y iverilog

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
iverilog is already the newest version (11.0-1.1).
0 upgraded, 0 newly installed, 0 to remove 

#Extension summary + prompt decisions
- Base tutorial example: sequence_detector
- Extension type: Parameterization (plus enable-gating)



Key prompt decisions and impact on RTL

1. Parameterize pattern length + bits (PAT_LEN, PATTERN, MAX_LEN) → implemented as a shift-register history and a masked compare, keeping the design synthesizable without dynamic arrays.

2. Define match timing as a 1-cycle pulse on the same cycle the last bit arrives → match is computed from the “next history” ({history[..], din}) to avoid a 1-cycle lag.

3. Add enable behavior → when enable=0, history holds and match=0, simplifying integration into larger systems and making behavior deterministic in the TB.

4. Verification decision: instantiate two parameter sets (4-bit and 6-bit patterns) in one TB and compare against a reference model → demonstrates parameterization works across configs, not just one case.

This is synthesizable RTL: single always block, no delays, no non-synth constructs in the DUT.

#Write seq_detector_param.v using Python

In [24]:
verilog_rtl = r"""
`timescale 1ns/1ps

// Parameterized sequence detector (synthesizable).
// Detects when the last PAT_LEN serial input bits equal PATTERN[ PAT_LEN-1 : 0 ].
// - enable: when 0, holds internal history (no shifting) and match output stays 0.
// - match: 1-cycle pulse when pattern is present after shifting in current bit.
module seq_detector_param #(
    parameter integer PAT_LEN = 4,           // pattern length (1..MAX_LEN)
    parameter integer MAX_LEN = 32,          // maximum supported pattern length
    parameter [MAX_LEN-1:0] PATTERN = {MAX_LEN{1'b0}}  // pattern bits in LSBs
)(
    input  wire clk,
    input  wire rst_n,     // active-low reset
    input  wire enable,
    input  wire din,       // serial input bit
    output reg  match
);

    reg [MAX_LEN-1:0] history;

    // Mask with LSB = 1 for PAT_LEN bits
    wire [MAX_LEN-1:0] mask = ({MAX_LEN{1'b1}} >> (MAX_LEN - PAT_LEN));

    always @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            history <= {MAX_LEN{1'b0}};
            match   <= 1'b0;
        end else if (enable) begin
            // shift in din into LSB side
            history <= {history[MAX_LEN-2:0], din};

            // compute match based on updated history (after shift)
            match <= ((({history[MAX_LEN-2:0], din}) & mask) == (PATTERN & mask));
        end else begin
            // hold history, force match low
            history <= history;
            match   <= 1'b0;
        end
    end

endmodule
"""

with open("seq_detector_param.v", "w") as f:
    f.write(verilog_rtl)

print("Wrote seq_detector_param.v")
!head -n 15 seq_detector_param.v


Wrote seq_detector_param.v

`timescale 1ns/1ps

// Parameterized sequence detector (synthesizable).
// Detects when the last PAT_LEN serial input bits equal PATTERN[ PAT_LEN-1 : 0 ].
// - enable: when 0, holds internal history (no shifting) and match output stays 0.
// - match: 1-cycle pulse when pattern is present after shifting in current bit.
module seq_detector_param #(
    parameter integer PAT_LEN = 4,           // pattern length (1..MAX_LEN)
    parameter integer MAX_LEN = 32,          // maximum supported pattern length
    parameter [MAX_LEN-1:0] PATTERN = {MAX_LEN{1'b0}}  // pattern bits in LSBs
)(
    input  wire clk,
    input  wire rst_n,     // active-low reset
    input  wire enable,


#Write tb_seq_detector_param.v using Python

In [26]:
tb = r"""
`timescale 1ns/1ps

module tb_seq_detector_param;

    reg clk = 0;
    always #5 clk = ~clk;

    reg rst_n  = 0;
    reg enable = 0;
    reg din    = 0;

    wire match4;
    wire match6;

    localparam integer MAX_LEN = 32;

    // PAT_LEN=4, pattern=4'b1011 in LSBs
    seq_detector_param #(
        .PAT_LEN(4),
        .MAX_LEN(MAX_LEN),
        .PATTERN(32'b0000_0000_0000_0000_0000_0000_0000_1011)
    ) dut4 (
        .clk(clk), .rst_n(rst_n), .enable(enable), .din(din), .match(match4)
    );

    // PAT_LEN=6, pattern=6'b110101 in LSBs
    seq_detector_param #(
        .PAT_LEN(6),
        .MAX_LEN(MAX_LEN),
        .PATTERN(32'b0000_0000_0000_0000_0000_0000_0011_0101)
    ) dut6 (
        .clk(clk), .rst_n(rst_n), .enable(enable), .din(din), .match(match6)
    );

    reg [MAX_LEN-1:0] ref_hist;
    reg ref_match4, ref_match6;

    function [MAX_LEN-1:0] mask_len(input integer L);
        begin
            mask_len = ({MAX_LEN{1'b1}} >> (MAX_LEN - L));
        end
    endfunction

    task step_and_check;
        input reg next_din;
        input reg next_enable;
        reg [MAX_LEN-1:0] next_hist;
        begin
            din    = next_din;
            enable = next_enable;

            @(posedge clk);

            if (!rst_n) begin
                ref_hist   = {MAX_LEN{1'b0}};
                ref_match4 = 1'b0;
                ref_match6 = 1'b0;
            end else if (enable) begin
                next_hist = {ref_hist[MAX_LEN-2:0], din};
                ref_hist  = next_hist;

                ref_match4 = ((next_hist & mask_len(4)) ==
                              (32'b0000_0000_0000_0000_0000_0000_0000_1011 & mask_len(4)));

                ref_match6 = ((next_hist & mask_len(6)) ==
                              (32'b0000_0000_0000_0000_0000_0000_0011_0101 & mask_len(6)));
            end else begin
                ref_match4 = 1'b0;
                ref_match6 = 1'b0;
            end

            if (match4 !== ref_match4) begin
                $display("FAIL match4 t=%0t DUT=%b REF=%b enable=%b din=%b hist=%b",
                         $time, match4, ref_match4, enable, din, ref_hist);
                $fatal(1);
            end

            if (match6 !== ref_match6) begin
                $display("FAIL match6 t=%0t DUT=%b REF=%b enable=%b din=%b hist=%b",
                         $time, match6, ref_match6, enable, din, ref_hist);
                $fatal(1);
            end
        end
    endtask

    integer i;

    initial begin
        $display("Starting TB...");

        ref_hist = {MAX_LEN{1'b0}};

        // reset for 2 cycles
        repeat (2) @(posedge clk);
        rst_n = 1;

        // Directed hit for 4-bit pattern 1011
        step_and_check(1, 1);
        step_and_check(0, 1);
        step_and_check(1, 1);
        step_and_check(1, 1);

        // Disable a few cycles
        for (i = 0; i < 5; i=i+1) begin
            step_and_check($random, 0);
        end

        // Random stress
        for (i = 0; i < 300; i=i+1) begin
            step_and_check($random, ($random % 4) != 0);
        end

        $display("PASS: All checks OK.");
        $finish;
    end

endmodule
"""

with open("tb_seq_detector_param.v", "w") as f:
    f.write(tb)

print("Wrote tb_seq_detector_param.v")
!head -n 20 tb_seq_detector_param.v


Wrote tb_seq_detector_param.v

`timescale 1ns/1ps

module tb_seq_detector_param;

    reg clk = 0;
    always #5 clk = ~clk;

    reg rst_n  = 0;
    reg enable = 0;
    reg din    = 0;

    wire match4;
    wire match6;

    localparam integer MAX_LEN = 32;

    // PAT_LEN=4, pattern=4'b1011 in LSBs
    seq_detector_param #(
        .PAT_LEN(4),


#Compile (iverilog)

In [27]:
!iverilog -g2012 -o sim.out tb_seq_detector_param.v seq_detector_param.v


#Run (vvp)

In [28]:
!vvp sim.out


Starting TB...
PASS: All checks OK.
